In [ ]:
osm = '/home/jasp/osm2lulc/osm_terceira.xml'

refraster = '/home/jasp/osm2lulc/lmt_terceira.shp'

result = '/mnt/d/osm2lulc2023/osm2lulc_v2/madeira_lulc.shp'

epsg = 5015

In [ ]:
import os
import datetime as dt

from glass.sql.db           import create_pgdb
from glass.it.db import osm_to_psql
from glass.pys.oss import mkdir, fprop
from glass.pys.tm import now_as_str
from glass.cons.osmtolulc import osm_features
from glass.gp.gen.sql import st_dissolve
from glass.prop.sql import row_num

In [ ]:
ws = mkdir(os.path.join(
    os.path.dirname(osm),
    'az_' + now_as_str()
), overwrite=True)

In [ ]:
polygons = 'multipolygons'
lines = 'lines'

### Pre-processamento

In [ ]:
osm_db = create_pgdb(fprop(osm, 'fn', forceLower=True), overwrite=True)
osm_db = osm_to_psql(osm, osm_db)

### Get OSM Features and classes

In [ ]:
osmfeat = osm_features()

osmfeat['whr'] = osmfeat['table'] + '.' + \
    osmfeat['key'] + "='" + osmfeat['value'] + "'"

### Module 1

In [ ]:
m1feat = osmfeat[(osmfeat.table == 'multipolygons') & (osmfeat.module == 1)]

time_a = dt.datetime.now().replace(microsecond=0)

polygons = 'multipolygons'
for cls in m1feat.cos2022.unique():
    df_cls = m1feat[m1feat.cos2022 == cls]
    
    whr_str = df_cls.whr.str.cat(sep=' OR ')
    
    nrow = row_num(
        osm_db, polygons,
        where=whr_str, api='psql'
    )
    
    if not nrow: continue
    
    shp = st_dissolve(
        osm_db, polygons,
        f'ST_Transform(wkb_geometry, {epsg})',
        os.path.join(ws, f'mod1_{str(cls)}.shp'),
        whrClause=whr_str,
        outTblIsFile=True,
        valascol={'lulc' : cls},
        geomout='geom', api='psql',
        multipart=None
    )

time_b = dt.datetime.now().replace(microsecond=0)

print(time_b - time_a)

### Module 2

In [ ]:
m2feat = osmfeat[
    (osmfeat.table == 'lines') &
    (osmfeat.module == 2) &
    (osmfeat.distbf > 0)
]

m2feat['caseaux'] = 'WHEN ' + m2feat['key'] + '=\'' + \
    m2feat['value'] + '\' THEN ' + m2feat['distbf']

time_a = dt.datetime.now().replace(microsecond=0)

nbuild = cnt_rows(
    osm_db, polygons,
    where="building IS NOT NULL",
    api='psql'
)

caselulc = 

time_b = dt.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
m2feat